# budget_from_ytd_to_html

E.Quinn 6/15/2020

Produces an html budget report from the 3/31/2020 YTD report

## Import standard python datascience packages

In [ ]:
import math
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cloudpickle
%matplotlib inline

In [ ]:
pd.set_option("display.max_rows",1000)
pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 50)

### Load RIDE UCOA labels 

In [ ]:
#Load data (deserialize)
with open('../UCOA_labels.pkl', 'rb') as handle:
    UCOA_labels = cloudpickle.load(handle)
    
help(UCOA_labels)

### Load EG acctcode labels 

In [ ]:
#Load data (deserialize)
with open('../EG_acct_codes.pkl', 'rb') as handle:
    EG_acct_codes = cloudpickle.load(handle)
help(EG_acct_codes)

### Read UCOA expense history

In [ ]:
pds = []

for fyear in np.arange(2018,2009,-1):
    fm1 = fyear-1
    path = '../RIDE/UCOA_files/Expense_' + str(fm1) + '_' + str(fyear) + '.csv'
    df1 = pd.read_csv(path)
    df1['fyear'] = fyear
    fyear = fyear - 1
    if ('District ID' in df1.columns):
        df1 = df1.rename(columns={'District ID': 'Dist No'})
    if ('Object' in df1.columns):
        df1 = df1.rename(columns={'Object': 'Obj',\
            'Revenue Object Description':'Object Description'})
    try:
        pds.append(df1.loc[df1['Dist No']==90.0])
    except KeyError:
        print("KeyError: ",path,df1.columns)
        
expdf = pd.concat(pds, axis=0, sort=False)
expdf = expdf.sort_values(['Obj', 'Loc','Prog','Func','Sub','JC'], \
        ascending=[True, True,True,True,True,True])
print(expdf.shape)
print(expdf.columns)

expdd = expdf.to_dict(orient='list')

### Code UCOA Loc from last two digits of ORG

In [ ]:
def get_loc(org):
    if (org % 100 == 5):
        return(3105.0)
    elif (org % 100 == 3):
        return(4103.0)
    elif (org % 100 == 8):
        return(3108.0)
    elif (org % 100 == 2):
        return(3102.0)
    elif (org % 100 == 7):
        return(3107.0)
    elif (org % 100 == 5):
        return(5106.0)
    else:
        return(np.NaN)

### Read the GL Journal Inquiry sheet from the 3/30/2020 YTD report

In [ ]:
gl = pd.read_csv("../School_no_journal_03 31 2020_GL_Journal_Inquiry.csv")
print(gl.shape)
print(gl.columns)
gldd = gl.to_dict(orient='list')
gl.head()

### Read the Account Detail sheet from the 3/30/2020 YTD report

In [ ]:
ad = pd.read_csv("../School_no_journal_03312020_account_detail.csv")
print(ad.shape)
print(ad.columns)
addd = ad.to_dict(orient='list')
ad.head()

### Loop through Account Detail dictionary and build objd dictionary for budget report

In [ ]:
objd = {'BUDGET TOTAL': 0.0,'ACTUALS TOTAL': 0.0}

level2 = 1

for i in np.arange(len(addd['OBJ'])):
    org = addd['ORG'][i]
    obj = addd['OBJ'][i]

    if (org != org):
        if (obj=='1'):
            objd['desc'] = addd['ACCOUNT DESCRIPTION'][i]
        elif (obj=='2'):
            objd[level2]['desc'] = addd['ACCOUNT DESCRIPTION'][i]
            level2+=1
        elif(obj!=obj):
            objd[level2][level3]['desc'] = addd['ACCOUNT DESCRIPTION'][i]
            level3+=1
    else:
        if level2 not in objd.keys():
            objd[level2]={'desc':'temp'}
            objd[level2]['BUDGET TOTAL'] = 0.0
            objd[level2]['ACTUALS TOTAL'] = 0.0
            level3=1
        if level3 not in objd[level2].keys():
            objd[level2][level3]={'desc':'temp'}
            objd[level2][level3]['BUDGET TOTAL'] = 0.0
            objd[level2][level3]['ACTUALS TOTAL'] = 0.0
            level4=1
        objd[level2][level3][level4]={}
        objd[level2][level3][level4]['ACCOUNT DESCRIPTION'] = addd['ACCOUNT DESCRIPTION'][i]
        objd[level2][level3][level4]['ORG'] = addd['ORG'][i]
        objd[level2][level3][level4]['OBJ'] = addd['OBJ'][i]
        objd[level2][level3][level4]['BUDGET'] = addd['REVISED BUD'][i]
        objd[level2][level3][level4]['ACTUAL'] = addd['EST ACTUALS'][i]
        objd[level2][level3]['BUDGET TOTAL'] += addd['REVISED BUD'][i]
        objd[level2][level3]['ACTUALS TOTAL'] += addd['EST ACTUALS'][i]
        objd[level2]['BUDGET TOTAL'] += addd['REVISED BUD'][i]
        objd[level2]['ACTUALS TOTAL'] += addd['EST ACTUALS'][i]
        objd['BUDGET TOTAL'] += addd['REVISED BUD'][i]
        objd['ACTUALS TOTAL'] += addd['EST ACTUALS'][i]
        level4+=1
            
            
objd.keys()

### Make a dictionary from the GL Journal Inquiry sheet

In [ ]:
gld = {}

for i in np.arange(len(gldd['ORG'])):
    org = gldd['ORG'][i]
    if org not in gld.keys():
        gld[org]={}
    obj = gldd['OBJECT'][i]
    if obj not in gld[org].keys():
        gld[org][obj] = {}
    gld[org][obj]['Fund']      = gldd['Fund'][i]
    gld[org][obj]['Location']  = gldd['Location'][i]
    gld[org][obj]['Location Description']  = UCOA_labels.get_label('Loc',gldd['Location'][i])
    gld[org][obj]['Function']  = gldd['Function'][i]
    gld[org][obj]['Function Description']  = UCOA_labels.get_label('Func',gldd['Function'][i])
    gld[org][obj]['Program']   = gldd['Program'][i]
    gld[org][obj]['Program Description']  = UCOA_labels.get_label('Prog',gldd['Program'][i])
    gld[org][obj]['Subject']   = gldd['Subject'][i]
    gld[org][obj]['Subject Description']  = UCOA_labels.get_label('Sub',gldd['Subject'][i])
    gld[org][obj]['Job Class'] = gldd['Job Class'][i]
    gld[org][obj]['Job Class Description']  = UCOA_labels.get_label('JC',gldd['Job Class'][i])
    gld[org][obj]['Object']    = gldd['Object'][i]
    gld[org][obj]['Object Description']  = UCOA_labels.get_label('Obj',gldd['Object'][i])
    gld[org][obj]['AMOUNT']    = float(gldd['AMOUNT'][i])
        
len(gld)

### Build a dictionary of Account Detail entries

* Fill in the UCOA codes and descriptions from the GL Journal Inquiry sheet by ORG/OBJ
* If there is no match in the GL Journal Inquiry sheet by ORG/OBJ, fill in the obj description from UCOA
* If there is no location code try to fill it in from the last two digits of ORG

In [ ]:
no_objd = {}                                                #dictionary for mismatches from GL Journal Inquiry

for lvl1 in objd.keys():                                    #loop through objd from Account Detail
    if isinstance(lvl1,int):                                #try to find matches by ORG/OBJ in GL Journal Inquiry
        for lvl2 in objd[lvl1].keys():                      #if we find a match, copy the UCOA codes
            if isinstance(lvl2,int):                        #and fill in the descriptions from UCOA 
                for lvl3 in objd[lvl1][lvl2].keys():
                    if isinstance(lvl3,int):
                        if 'ORG' in objd[lvl1][lvl2][lvl3].keys():          #make sure ORG is there
                            org = objd[lvl1][lvl2][lvl3]['ORG']             #also get OBJ
                            obj = int(objd[lvl1][lvl2][lvl3]['OBJ'])        #try to find ORG/OBJ in GL dictionary
                            if org not in gld.keys():                       #look up the UCOA codes by ORG  
                                egc = EG_acct_codes.get_eg_acct_UCOA(org)   #
                                if (len(egc) > 0):                          #it's probably not there
                                    print(org,egc)                          #but if it is print it (fix this later)
                                loc = get_loc(org)                          #see if we can get UCOA Loc from ORG
                                if (loc==loc):                              #if we can, fill it in 
                                    objd[lvl1][lvl2][lvl3]['Location'] = loc
                                    objd[lvl1][lvl2][lvl3]['Location Description'] = \
                                        UCOA_labels.get_label('Loc',loc)
                                objd[lvl1][lvl2][lvl3]['Object Description'] = \
                                    UCOA_labels.get_label('Obj',obj)
                                no_objd[org] = {}                           #save GL mismatches
                                no_objd[org][obj] = objd[lvl1][lvl2][lvl3]

                            else:                                           #if ORG/OBJ is there, copy UCOA codes
                                if obj in gld[org].keys():
                                    objd[lvl1][lvl2][lvl3]['Fund'] = gld[org][obj]['Fund']
                                    objd[lvl1][lvl2][lvl3]['Location'] = gld[org][obj]['Location']
                                    objd[lvl1][lvl2][lvl3]['Location Description'] = \
                                        gld[org][obj]['Location Description']
                                    objd[lvl1][lvl2][lvl3]['Function'] = gld[org][obj]['Function']
                                    objd[lvl1][lvl2][lvl3]['Function Description'] = \
                                        gld[org][obj]['Function Description']
                                    objd[lvl1][lvl2][lvl3]['Program'] = gld[org][obj]['Program']
                                    objd[lvl1][lvl2][lvl3]['Program Description'] = \
                                        gld[org][obj]['Program Description']
                                    objd[lvl1][lvl2][lvl3]['Subject'] = gld[org][obj]['Subject']
                                    objd[lvl1][lvl2][lvl3]['Subject Description'] = \
                                        gld[org][obj]['Subject Description']
                                    objd[lvl1][lvl2][lvl3]['Job Class'] = gld[org][obj]['Job Class']
                                    objd[lvl1][lvl2][lvl3]['Job Class Description'] = \
                                        gld[org][obj]['Job Class Description']
                                    objd[lvl1][lvl2][lvl3]['Object'] = gld[org][obj]['Object']
                                    objd[lvl1][lvl2][lvl3]['Object Description'] = \
                                        gld[org][obj]['Object Description']
                                    objd[lvl1][lvl2][lvl3]['AMOUNT'] = gld[org][obj]['AMOUNT']   #copy AMOUNT
        

### historical_budget function

* Creates a budget in the current format from UCOA expense history

In [ ]:
def historical_budget(objd,fyear):
    
    print(fyear)
    
    h = {}

    for key in objd.keys():
        if (not isinstance(key,int)):
            if (isinstance(objd[key],float)):
                h[key]=0.0
            else:
                h[key] = objd[key]
        else:
            h[key] = {}
            for lvl2 in objd[key].keys():
                if (not isinstance(lvl2,int)):
                    if (isinstance(objd[key][lvl2],float)):
                        h[key][lvl2]=0.0
                    else:
                        h[key][lvl2] = objd[key][lvl2]
                else:
                    h[key][lvl2] = {}
                    for lvl3 in objd[key][lvl2].keys():
                        if (isinstance(lvl3,int)):
                            h[key][lvl2]['OBJ']=objd[key][lvl2][lvl3]['OBJ']
                            obj = float(objd[key][lvl2][lvl3]['OBJ'])
                            h[key][lvl2]['desc'] = UCOA_labels.get_label('Obj',obj)
                            h[key][lvl2]['BUDGET TOTAL'] = 0.0
                            h[key][lvl2]['ACTUALS TOTAL'] = 0.0

    for key in h.keys():
        if (isinstance(key,int)):
            for lvl2 in h[key].keys():
                lno = 1
                if (isinstance(lvl2,int)):
                    obj = float(h[key][lvl2]['OBJ'])
                    for i in np.arange(len(expdd['Obj'])):
                        if (expdd['fyear'][i]==fyear):
                            if (obj == expdd['Obj'][i]):
                                h[key][lvl2][lno] = {}
                                h[key][lvl2][lno]['Fund'] = expdd['Fund'][i]
                                h[key][lvl2][lno]['Prog'] = expdd['Prog'][i]
                                h[key][lvl2][lno]['Func'] = expdd['Func'][i]
                                h[key][lvl2][lno]['Sub'] = expdd['Sub'][i]
                                h[key][lvl2][lno]['JC'] = expdd['JC'][i]
                                h[key][lvl2][lno]['Loc'] = expdd['Loc'][i]
                                h[key][lvl2][lno]['Location Description'] =\
                                    UCOA_labels.get_label('Loc',float(expdd['Loc'][i]))
                                h[key][lvl2][lno]['Program Description'] =\
                                    UCOA_labels.get_label('Prog',float(expdd['Prog'][i]))
                                h[key][lvl2][lno]['Function Description'] =\
                                    UCOA_labels.get_label('Func',float(expdd['Func'][i]))
                                h[key][lvl2][lno]['Subject Description'] =\
                                    UCOA_labels.get_label('Sub',float(expdd['Sub'][i]))
                                h[key][lvl2][lno]['Job Class Description'] =\
                                    UCOA_labels.get_label('JC',float(expdd['JC'][i]))
                                h[key][lvl2][lno]['Actual'] = expdd['Actual'][i]
                                h[key][lvl2][lno]['Budget'] = expdd['Budget'][i]
                                h[key][lvl2]['BUDGET TOTAL'] += h[key][lvl2][lno]['Budget']
                                h[key][lvl2]['ACTUALS TOTAL'] += h[key][lvl2][lno]['Actual']
                                lno+=1
                    h[key]['BUDGET TOTAL'] += h[key][lvl2]['BUDGET TOTAL']
                    h[key]['ACTUALS TOTAL'] += h[key][lvl2]['ACTUALS TOTAL']
            h['BUDGET TOTAL'] += h[key]['BUDGET TOTAL']
            h['ACTUALS TOTAL'] += h[key]['ACTUALS TOTAL']
                                
    f= open('../www/fy' + str(fyear) + '_budget.html','w')

    f.write('<html><body><p>\n')
    f.write('<h1><b>East Greenwich School Department FY' + str(fyear) + ' Budget</b></h1><hr>\n')
    f.write('<h2><b>Category Level Summary</b></h2><hr>\n')
    f.write('<table border="1" width="95%">\n')
    f.write('<tr align="center" bgcolor="#00C0C0"><td><b>Category</b></td>' +\
        '<td><b>Budget Total</b></td>' +\
        '<td><b>Actuals Total</b></td>' +\
        '<td><b>Variance</b></td></tr>\n')

    for lvl1 in h.keys():
        if isinstance(lvl1,int):
            if 'desc' in h[lvl1].keys():
                desc = h[lvl1]['desc']
            else:
                print('no desc for',lvl1,lvl2,lvl3,h[lvl1])
            rbt = h[lvl1]['BUDGET TOTAL']
            eat = h[lvl1]['ACTUALS TOTAL']
            var = rbt-eat
            f.write('<tr><td align="left"><a href="#lvl1_' + str(lvl1) + '">' + desc + '</a></td>'+\
                '<td align="right">' + '${:,.0f}'.format(rbt) + '</td>' +\
                '<td align="right">' + '${:,.0f}'.format(eat) + '</td>' +\
                '<td align="right">' + '${:,.0f}'.format(var) + '</td></tr>\n')
    desc = h['desc']
    rbt = h['BUDGET TOTAL']
    eat = h['ACTUALS TOTAL']
    var = rbt-eat
    f.write('<tr><td align="center"><b>' + desc + '</b></td>'+\
        '<td align="right">' + '${:,.0f}'.format(rbt) + '</td>' +\
        '<td align="right">' + '${:,.0f}'.format(eat) + '</td>' +\
        '<td align="right">' + '${:,.0f}'.format(var) + '</td></tr>\n')
    f.write('</table><p><hr>\n')
    
    f.write('<html><body><p>\n')
    f.write('<h2><b>Object Code within Category</b></h2><hr>\n')
    f.write('<table border="1" width="95%">\n')

    for lvl1 in h.keys():
        if isinstance(lvl1,int):
            cdesc = h[lvl1]['desc']
            crbt = h[lvl1]['BUDGET TOTAL']
            ceat = h[lvl1]['ACTUALS TOTAL']
            cvar = crbt-ceat
            f.write('<a name="lvl1_' + str(lvl1) + '"></a><hr>\n')
            f.write('<tr><td><b>' + cdesc + '</b></td></tr>')
            f.write('<tr><td>\n')
            f.write('<table border="1" width="95%">')
            f.write('<tr align="center" bgcolor="#00C0C0"><td><b>Object Code</b></td>' +\
                '<td><b>Budget Total</b></td>' +\
                '<td><b>Actuals Total</b></td>' +\
                '<td><b>Variance</b></td></tr>\n')
            n_levels=0
            for lvl2 in h[lvl1].keys():
                if isinstance(lvl2,int):
                    n_levels+=1
                    desc = h[lvl1][lvl2]['desc']
                    rbt  = h[lvl1][lvl2]['BUDGET TOTAL']
                    eat  = h[lvl1][lvl2]['ACTUALS TOTAL']
                    var = rbt-eat
                    f.write('<tr><td align="left"><a href="#lvl1_' + str(lvl1) +\
                        '_lvl2_' + str(lvl2) + '">' + desc + '</a></td>'+\
                        '<td align="right">' + '${:,.0f}'.format(rbt) + '</td>' +\
                        '<td align="right">' + '${:,.0f}'.format(eat) + '</td>' +\
                        '<td align="right">' + '${:,.0f}'.format(var) + '</td></tr>\n')
            if (n_levels > 1):
                f.write('<tr><td align="center"><b>' + cdesc + '</b></td>'+\
                    '<td align="right">' + '${:,.0f}'.format(crbt) + '</td>' +\
                    '<td align="right">' + '${:,.0f}'.format(ceat) + '</td>' +\
                    '<td align="right">' + '${:,.0f}'.format(cvar) + '</td></tr>\n')
                f.write('</table><p><hr>\n')
            
            f.write('</table>\n')
            
    f.write('<html><body><p>\n')
    f.write('<h2><b>Object Code within Category</b></h2><hr>\n')
    f.write('<table border="1" width="95%">\n')

    for lvl1 in h.keys():
        if isinstance(lvl1,int):
            for lvl2 in h[lvl1].keys():
                if isinstance(lvl2,int):
                    cdesc = h[lvl1][lvl2]['desc']
                    crbt = h[lvl1][lvl2]['BUDGET TOTAL']
                    ceat = h[lvl1][lvl2]['ACTUALS TOTAL']
                    cvar = crbt-ceat
                    obj  = h[lvl1][lvl2]['OBJ']
                    desc = UCOA_labels.get_label('Obj',float(obj))
                    f.write('<a name="lvl1_' + str(lvl1) + '_lvl2_' + str(lvl2) + '"></a><hr>\n')
                    f.write('<tr><td><b>' + obj + ' ' + cdesc + '</b></td></tr>')
                    f.write('<tr><td>\n')
                    f.write('<table border="1" width="95%">')
                    f.write('<tr align="center" bgcolor="#00C0C0">' +\
                        '<td><b>Location</b></td>' +\
                        '<td><b>Program</b></td>' +\
                        '<td><b>Function</b></td>' +\
                        '<td><b>Job Class</b></td>' +\
                        '<td><b>Subject</b></td>' +\
                        '<td><b>Budget</b></td>' +\
                        '<td><b>Actuals</b></td>' +\
                        '<td><b>Variance</b></td></tr>\n')
                    n_levels=0
                    for lvl3 in h[lvl1][lvl2].keys():
                        if isinstance(lvl3,int):
                            n_levels+=1
                            locd = '&nbsp;'
                            if 'Location Description' in h[lvl1][lvl2][lvl3].keys():
                                locd = h[lvl1][lvl2][lvl3]['Location Description']
                            progd = '&nbsp;'
                            if 'Program Description' in h[lvl1][lvl2][lvl3].keys():
                                progd = h[lvl1][lvl2][lvl3]['Program Description']
                            funcd = '&nbsp;'
                            if 'Function Description' in h[lvl1][lvl2][lvl3].keys():
                                funcd = h[lvl1][lvl2][lvl3]['Function Description']
                            jcd = '&nbsp;'
                            if 'Job Class Description' in h[lvl1][lvl2][lvl3].keys():
                                jcd = h[lvl1][lvl2][lvl3]['Job Class Description']
                            subd = '&nbsp;'
                            if 'Subject Description' in h[lvl1][lvl2][lvl3].keys():
                                subd = h[lvl1][lvl2][lvl3]['Subject Description']

                            rbt  = h[lvl1][lvl2][lvl3]['Budget']
                            eat  = h[lvl1][lvl2][lvl3]['Actual']
                            var = rbt-eat
                            f.write('<tr><td align="left">' + locd + '</td>'+\
                                '<td align="center">' + progd + '</td>' +\
                                '<td align="center">' + funcd + '</td>' +\
                                '<td align="center">' + jcd + '</td>' +\
                                '<td align="center">' + subd + '</td>' +\
                                '<td align="right">' + '${:,.0f}'.format(rbt) + '</td>' +\
                                '<td align="right">' + '${:,.0f}'.format(eat) + '</td>' +\
                                '<td align="right">' + '${:,.0f}'.format(var) + '</td></tr>\n')
                    if (n_levels > 1):
                        f.write('<tr><td align="center" colspan="5"><b>' + cdesc + '</td>'+\
                            '<td align="right">' + '${:,.0f}'.format(crbt) + '</td>' +\
                            '<td align="right">' + '${:,.0f}'.format(ceat) + '</td>' +\
                            '<td align="right">' + '${:,.0f}'.format(cvar) + '</b></td></tr>\n')
                    f.write('</table></body></html><hr>\n')
            
            f.write('</table>\n')
    f.close()
    return

                                

In [ ]:
for fyear in np.arange(2018,2009,-1):
    historical_budget(objd,fyear)